In [ ]:

import torch.nn as nn

from ast import Sub
#from types import NoneType
from typing import ForwardRef
import torch
import pandas as pd
import time
import re
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.transforms.functional as TF
import argparse
import numpy as np
import scipy.io
import requests
import math
from PIL import *
from os import listdir
import os
from glob import glob
from os.path import isfile, isdir, join, splitext
import importlib
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score, cohen_kappa_score
from sklearn.model_selection import StratifiedGroupKFold, KFold
from datetime import datetime
import os
import glob
import shutil

from torch.optim.swa_utils import *
from performance import *


def get_Kfold_index(dataset,n_splits=10,random_state=None):
    #Statified group k fold based on number of stroke ROIs
    skf = StratifiedGroupKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    labels = np.stack(dataset.label_list,axis=0)
    subj_idx = np.arange(len(dataset))
    subj_idx = np.tile(subj_idx,(1,*labels.shape[1:])).flatten()
    fold_idx = []
    fold_iter = skf.split(labels.flatten(), labels.flatten(), subj_idx)
    for train_idx, test_idx in fold_iter:
        subj_train = np.unique(subj_idx[train_idx])
        subj_test = np.unique(subj_idx[test_idx])
        fold_idx.append((subj_train,subj_test))
    return fold_idx


# initialize dataset
main_data =  TorchDataset(image_dir = 'D:/File_X/PHD/ASPECT_class/data/wholebrain_CT_pytorch_dataset_ASPECTSCrop_DS',
        repeat=1,augment=True,z_crop=False,dtype=torch.float32,pre_align=False,
        aug_Rrange=15, aug_Trange=0.02,
        hflip=False,vflip=False)    
print("Main dataset:\n", main_data)

# initialize Cv index
kfold_idx = get_Kfold_index(main_data,n_splits=2,random_state=None)
#skf = KFold(n_splits=args.num_folds,shuffle=True,random_state=args.seed)
K = 0
num_test_aug = 1
for train_index, test_index in kfold_idx:
    # print('train_index: ', len(train_index),
    #       ' test_index: ', len(test_index))
    K += 1
    if K < 1: # Skip finished fold
        continue
    print('=====================\tFold %d\t======================='%K)

    # initialize dataset
    train_data = copy.deepcopy(main_data)
    valid_data = copy.deepcopy(main_data)
    train_data.filter(train_index)
    valid_data.filter(test_index)
    if  num_test_aug ==1:
        valid_data.no_augment()
    valid_data.n_augs = num_test_aug
    
    print("Training dataset:\n", train_data)
    print("Validation dataset:\n", valid_data)
    
#Change path
os.listdir('data/wholebrain_CT_pytorch_dataset_ASPECTSCrop_DS')
mat_list = glob.glob('data/wholebrain_CT_pytorch_dataset_ASPECTSCrop_DS/*.mat')
len(mat_list)

presentOnDisk_set = [os.path.split(p)[-1] for p in mat_list]

for ind in kfold_idx[0][0]:
    # print(ind)
    original = r'data/wholebrain_CT_pytorch_dataset_ASPECTSCrop_DS/' + presentOnDisk_set[ind]
    target = r'data/Train1/' + presentOnDisk_set[ind]
    
    shutil.copyfile(original, target)

for ind in kfold_idx[0][1]:
    # print(ind)
    original = r'data/wholebrain_CT_pytorch_dataset_ASPECTSCrop_DS/' + presentOnDisk_set[ind]
    target = r'data/Train2/' + presentOnDisk_set[ind]
    
    shutil.copyfile(original, target)


#2 fold dataset
main_data1 =  TorchDataset(image_dir = 'D:/File_X/PHD/ASPECT_class/data/Train1',
        repeat=1,augment=True,z_crop=False,dtype=torch.float32,pre_align=False,
        aug_Rrange=15, aug_Trange=0.02,
        hflip=False,vflip=False)    
print("Main dataset 1:\n", main_data1)

main_data2 =  TorchDataset(image_dir = 'D:/File_X/PHD/ASPECT_class/data/Train2',
        repeat=1,augment=True,z_crop=False,dtype=torch.float32,pre_align=False,
        aug_Rrange=15, aug_Trange=0.02,
        hflip=False,vflip=False)    
print("Main dataset 2:\n", main_data2)



In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.utils.data as U

x = U.random_split(range(10), [3, 7], generator=torch.Generator().manual_seed(42))
x

In [70]:
import torch.utils.data as data
from torchvision import datasets


# Load data set
train_data_path = '/home/yingchihlin/Documents/Code/AISD/AISD.train'
id_list = os.listdir(train_data_path)
train_set = sorted(id_list)

# Before
# print('Train data set:', len(train_set))

# Random split
train_set_size = int(len(train_set) * 0.85)
valid_set_size = len(train_set) - train_set_size
# train_set, valid_set = data.random_split(train_set, [train_set_size, valid_set_size])

# # After
# print('='*30)
# print('Train data set:', len(train_set))
# print('Valid data set:', len(valid_set))

# valid_set
print(train_set_size, valid_set_size)


294 52


In [71]:
import random

val_set = random.sample(list(train_set), valid_set_size)
# print(val_list)

# train_data = list(train_set).remove('0538833')
[train_set.remove(x) for x in val_set]

# subj_names = [os.path.split(image_paths[x])[0][-7:] for x in range(image_num)]

print(len(train_set), len(val_set))

294 52


In [87]:
#train
import glob
from logging import root
import os
import shutil


train_image_paths = [glob.glob('/home/yingchihlin/Documents/Code/AISD/AISD.train/{}'.format(uid))[0] for uid in train_set]
val_image_paths = [glob.glob('/home/yingchihlin/Documents/Code/AISD/AISD.train/{}'.format(uid))[0] for uid in val_set]

root_path = os.getcwd()
# print(root_path)

train_data_path = os.path.join(root_path, 'train')
val_data_path = os.path.join(root_path, 'validation')

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)

if not os.path.exists(val_data_path):
    os.makedirs(val_data_path)

# os.path.split(train_image_paths[0])[1]
# train_image_paths[0]

# file = train_image_paths[0]
# target = os.path.join(train_data_path, os.path.split(file)[1])
# print(target)
# shutil.copytree(file, target)


for file in train_image_paths:

    target = os.path.join(train_data_path, os.path.split(file)[1])
    shutil.copytree(file, target)

for file in val_image_paths:

    target = os.path.join(val_data_path, os.path.split(file)[1])
    shutil.copytree(file, target)